# How to grid search ARIMA and SARIMA models hyperparameters

<p style="text-align: justify">In this notebook, an exmaple of grid search on ARIMA and SARIMA model parameters is proposed. This notebook is complementary of the 1st one <a href="https://github.com/DavidCico/Univariate-time-series-analysis-of-cryptocurrency-data-with-ARIMA-and-SARIMA-and-hypergrid-search/blob/master/Univariate_analysis_classic_methods.ipynb">Univariate_analysis_classic_methods</a>, and follows the similar procedure in terms of data loading, definition of the testing and training sets, and also for the implementation of the ARIMA and SARIMA models. To avoid repetition, the reader should read this <a href="https://github.com/DavidCico/Univariate-time-series-analysis-of-cryptocurrency-data-with-ARIMA-and-SARIMA-and-hypergrid-search/blob/master/Univariate_analysis_classic_methods.ipynb">notebook</a> first before the current one.


<p style="text-align: justify">The ARIMA model for time series analysis and forecasting can be tricky to configure. There are 3 parameters that require estimation by iterative trial and error from reviewing diagnostic plots and using 40-year-old heuristic rules. We can automate the process of evaluating a large number of hyperparameters for the ARIMA model by using a grid search procedure.</p>

<p style="text-align: justify">The Seasonal Autoregressive Integrated Moving Average, or SARIMA, model is an approach for modeling univariate time series data that may contain trend and seasonal components. It is an effective approach for time series forecasting, although it requires careful analysis and domain expertise in order to configure the seven or more model hyperparameters. An alternative approach to configuring the model that makes use of fast and parallel modern hardware is to grid search a suite of hyperparameter configurations in order to discover what works best. Often, this process can reveal non-intuitive model configurations that result in lower forecast error than those configurations specified through careful analysis.</p>

<p style="text-align: justify">In this notebook, you will discover how to develop a framework for grid searching all of the ARIMA and SARIMA model hyperparameters for univariate time series forecasting.</p>


## SARIMA (and ARIMA) for Time Series Forecasting 

<p style="text-align: justify">Seasonal Autoregressive Integrated Moving Average, SARIMA or Seasonal ARIMA, is an extension of ARIMA that explicitly supports univariate time series data with a seasonal component.</p>
<p style="text-align: justify">It adds three new hyperparameters to specify the autoregression (AR), differencing (I), and moving average (MA) for the seasonal component of the series, as well as an additional parameter for the period of the seasonality.</p>
<blockquote><p>A seasonal ARIMA model is formed by including additional seasonal terms in the ARIMA [&#8230;] The seasonal part of the model consists of terms that are very similar to the non-seasonal components of the model, but they involve backshifts of the seasonal period.</p></blockquote>
<p style="text-align: justify">&#8212; Page 242, <a href="https://amzn.to/2xlJsfV">Forecasting: principles and practice</a>, 2013.</p>
<p style="text-align: justify">Configuring a SARIMA requires selecting hyperparameters for both the trend and seasonal elements of the series.</p>
<p style="text-align: justify">There are three trend elements that require configuration.</p>
<p style="text-align: justify">They are the same as the ARIMA model; specifically:</p>
<ul>
<li><strong>p</strong>: Trend autoregression order.</li>
<li><strong>d</strong>: Trend difference order.</li>
<li><strong>q</strong>: Trend moving average order.</li>
</ul>
<p style="text-align: justify">There are four seasonal elements that are not part of ARIMA that must be configured; they are:</p>
<ul>
<li><strong>P</strong>: Seasonal autoregressive order.</li>
<li><strong>D</strong>: Seasonal difference order.</li>
<li><strong>Q</strong>: Seasonal moving average order.</li>
<li><strong>m</strong>: The number of time steps for a single seasonal period.</li>
</ul>
<p style="text-align: justify">Together, the notation for a SARIMA model is specified as:
    
    SARIMA(p,d,q)(P,D,Q)m

<p style="text-align: justify">The SARIMA model can subsume the ARIMA, ARMA, AR, and MA models via model configuration parameters.</p>

<p style="text-align: justify">The trend and seasonal hyperparameters of the model can be configured by analyzing autocorrelation and partial autocorrelation plots, and this can take some expertise.</p>

<p style="text-align: justify">An alternative approach is to grid search a suite of model configurations and discover which configurations work best for a specific univariate time series.</p>

## Grid search framework

<p style="text-align: justify">In this section, we will develop a framework for grid searching SARIMA model hyperparameters for a given univariate time series forecasting problem.

<p style="text-align: justify">We use the implementation of <a href="http://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_model.ARIMA.html">ARIMA</a> and <a href="http://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html">SARIMA</a> provided by the statsmodels library.

<p style="text-align: justify">This model has hyperparameters that control the nature of the model performed for the series, trend and seasonality, specifically:</p>

<ul>
<li><strong>order</strong>: A tuple p, d, and q parameters for the modeling of the trend (only parameters required for ARIMA).</li>
<li><strong>sesonal_order</strong>: A tuple of P, D, Q, and m parameters for the modeling the seasonality</li>
<li><strong>trend</strong>: A parameter for controlling a model of the deterministic trend as one of &#8216;n&#8217;,&#8217;c&#8217;,&#8217;t&#8217;,&#8217;ct&#8217; for no trend, constant, linear, and constant with linear trend, respectively.</li>
</ul>
<p style="text-align: justify">If you know enough about your problem to specify one or more of these parameters, then you should specify them. If not, you can try grid searching these parameters.</p>

<p style="text-align: justify">The grid search framework is defined at first in a similar way as in the <a href="https://github.com/DavidCico/Univariate-time-series-analysis-of-cryptocurrency-data-with-ARIMA-and-SARIMA-and-hypergrid-search/blob/master/Univariate_analysis_classic_methods.ipynb">first notebook</a>, concerning the split of the data, the general implementation of the statistic models, as well as the evaluation metric. Here, we repeat of the content at first, to set the framework before introducing the functions for grid search.</p>


### Dataset split

<p style="text-align: justify">The data in a given dataset will be divided into standard weeks. These are weeks that begin on a Monday and end on a Sunday.</p>

<p style="text-align: justify">This is a realistic and useful way for using the chosen framing of the model, where the price for the week ahead can be predicted. It is also helpful with modeling, where models can be used to predict a specific day (e.g. Wednesday) or the entire sequence.</p>

<p style="text-align: justify">We will split the data into standard weeks, working backwards from the test dataset. This gives 178 weeks of data for the training set and 102 weeks (714 days) for the testing set.</p>

<p style="text-align: justify">The function <em>split_dataset()</em> below splits the daily data into train and test sets and organizes each into standard weeks. The "<i>n_test</i>" argument corresponds to the number of days (714 in this study), to cut the data backwards.</p>

<p style="text-align: justify">Specific row offsets are used to split the data using knowledge of the dataset. The split datasets are then organized into weekly data using the NumPy <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.split.html">split() function</a>.</p>

<p style="text-align: justify">The <em>to_series()</em> function will also take the multivariate data divided into weekly windows and will return a single univariate time series.</p>


In [2]:
# split a univariate dataset into train/test sets
def split_dataset(data, n_test):
    # split into standard weeks
    train, test = data[0:-n_test], data[-n_test:]
    # restructure into windows of weekly data
    train = np.array(np.split(train, len(train)/7))
    test = np.array(np.split(test, len(test)/7))
    return train, test

# convert windows of weekly multivariate data into a series of closing price
def to_series(data):
    # extract just the price of XRP from each week
    series = [week[:, 0] for week in data]
    # flatten into a single series
    series = np.array(series).flatten()
    return series

### Autoregression Models

<p style="text-align: justify">The Statsmodels library provides multiple ways of developing an AR model, such as using the AR, ARMA, ARIMA, and SARIMAX classes.</p>

<p style="text-align: justify">We will use the <a href="http://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_model.ARIMA.html">ARIMA implementation</a> as it allows for easy expandability into differencing and moving average. </p>

<p style="text-align: justify">The <em>arima_forecast()</em> function defined below implements the procedure for doing a prediction with an ARIMA model for 7 days in a row. </p>

In [3]:
# Arima forecast for weekly prediction
def arima_forecast(history, arima_order):
    # convert history into a univariate series
    series = to_series(history)
    # define the model
    model = ARIMA(series, order=arima_order)
    # fit the model
    model_fit = model.fit(disp=False)
    # make forecast
    yhat = model_fit.predict(len(series), len(series)+6)
    return yhat

<p style="text-align: justify">The seasonal ARIMA model <a href="https://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html">SARIMA</a>, can also be implemented in a similar way as the ARIMA model. The <em>Sarima_forecast</em> function below implements the same procedure as above, with the <em>"config"</em> argument defining the configuration of the chosen SARIMA model. The seasonal ARIMA model takes more parameters than the regular ARIMA model, to characterize some seasonal trends that might be present inside the dataset.</p>

In [4]:
# Sarima forecast for weekly prediction
def Sarima_forecast(history, config):
    order, sorder, trend = config
    # convert history into a univariate series
    series = to_series(history)
    # define model
    model = SARIMAX(series, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
    # fit model
    model_fit = model.fit(disp=False)
    # make one step forecast
    yhat = model_fit.predict(len(series), len(series)+6)
    return yhat

### Evaluation metric

<p style="text-align: justify">A forecast will be comprised of seven values, one for each day of the week ahead.</p>

<p style="text-align: justify">The performance metric for this problem will be the <a href="https://en.wikipedia.org/wiki/Root-mean-square_deviation">Root Mean Squared Error (RMSE)</a> for each lead time from day 1 to day 7. In this way, we can see how the chosen algorithms perform on the predictions at a particular day of the week. The cryptocurrency market is quite volatile, and may have a different behaviour depending on the period of the week (weekdays or weekend for instance).</p>
<p style="text-align: justify">As a short-cut, it may be useful to summarize the performance of a model using a single score in order to help in model selection. One possible score that could be used would be the RMSE across all forecast days.</p>

<p style="text-align: justify">The function <i>evaluate_forecasts()</i> below will implement this behavior and return the performance of a model based on multiple seven-day forecasts.</p>

In [5]:
# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
    scores = list()
    # calculate an RMSE score for each day
    for i in range(actual.shape[1]):
        # calculate mse
        mse = mean_squared_error(actual[:, i], predicted[:, i])
        # calculate rmse
        rmse = sqrt(mse)
        # store
        scores.append(rmse)
    # calculate overall RMSE
    s = 0
    for row in range(actual.shape[0]):
        for col in range(actual.shape[1]):
            s += (actual[row, col] - predicted[row, col])**2
    score = sqrt(s / (actual.shape[0] * actual.shape[1]))
    return score, scores

### Walk-forward validation

<p style="text-align: justify">Models will be evaluated using a scheme called <a href="https://en.wikipedia.org/wiki/Walk_forward_optimization">walk-forward validation</a>.</p>

<p style="text-align: justify">This is where a model is required to make a one week prediction, then the actual data for that week is made available to the model so that it can be used as the basis for making a prediction on the subsequent week. This is both realistic for how the model may be used in practice and beneficial to the models allowing them to make use of the best available data.</p>

<p style="text-align: justify">We can demonstrate this below with separation of input data and output/predicted data.</p>

    Input, 						Predict
    [Week1]						Week2
    [Week1 + Week2]				Week3
    [Week1 + Week2 + Week3]		Week4

<p style="text-align: justify">The walk-forward validation approach to evaluating predictive models on this dataset is implement below, named <em>evaluate_model()</em>.</p>

<p style="text-align: justify">The name of a function is provided for the model as the argument "<em>model_func</em>";. This function is responsible for defining the model, fitting the model on the training data, and making a one-week forecast.</p>

<p style="text-align: justify">The forecasts made by the model are then evaluated against the test dataset using the previously defined <em>evaluate_forecasts()</em> function.</p>

In [6]:
def evaluate_model(model_func, train, test, *args):
    #history of weekly data
    history = [x for x in train]
    #walk forward validation
    predictions = list()
    for i in range(len(test)):
    #weekly prediction
        y_hat_seq = model_func(history, *args)
    #store the preditions
        predictions.append(y_hat_seq)
    #update history data
        history.append(test[i,:])
    predictions = np.array(predictions)
    # evaluate predictions days for each week
    score, scores = evaluate_forecasts(test[:, :, 0], predictions)
    return score, scores, predictions

### Evaluating performance of a ARIMA family models

<p style="text-align: justify">We can call <em>evaluate_model()</em> repeatedly for a specific model and different lists of model configurations.</p>

<p style="text-align: justify">One possible issue is that some combinations of model configurations may not be called for the model and will throw an exception, e.g. specifying some but not all aspects of the seasonal structure in the data.</p>

<p style="text-align: justify">Further, some models may also raise warnings on some data, e.g. from the linear algebra libraries called by the statsmodels library.</p>

<p style="text-align: justify">We can trap exceptions and ignore warnings during the grid search by wrapping all calls to <em>evaluate_model()</em> with a try-except and a block to ignore warnings. We can also add debugging support to disable these protections in the case we want to see what is really going on. Finally, if an error does occur, we can return a None result, otherwise we can print some information about the skill of each model evaluated. This is helpful when a large number of models are evaluated.</p>

<p style="text-align: justify">The <em>evaluate_arima_family_scores</em> function below implements this and returns a tuple of (key and result), where the key is a string version of the tested model configuration.</p>

In [7]:
# Return score on ARIMA family model, for model assessment
def evaluate_arima_family_scores(func, name, train, test, order, debug = False):
    score = None
    # show all warnings and fail on exception if debugging
    if debug:
        score, scores, predictions = evaluate_model(func, train, test, order) #evaluate particular model with walk-forward validation
    else:
        # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")
                score, scores, predictions = evaluate_model(func, train, test, order)
        except:
            score = None 
    # check for an interesting result
    if score is not None: # won't print model configurations that returned nothing
        print(name + '%s RMSE=%.3f' % (order,score))
    return (order, score)

<p style="text-align: justify">Next, we need a loop to test a list of different model configurations.</p>

<p style="text-align: justify">This is the main function that drives the grid search process and will call the <em>evaluate_arima_family_scores()</em> function for model configuration of the selected model.</p>

<p style="text-align: justify">We can dramatically speed up the grid search process by executing this process in parallel. One way to do that is to use the <a href="https://pypi.org/project/joblib/">Joblib library</a>.</p>

<p style="text-align: justify">We can define a Parallel object with the number of cores to use and set it to the number of scores detected in our hardware.</p>

```python
executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
```

<p style="text-align: justify">We can then can then create a list of tasks to execute in parallel, which will be one call to the <em>evaluate_arima_family_scores()</em> function for each model configuration we have.</p>

```python
tasks = (delayed(evaluate_arima_family_scores)(func, name, order) for order in orders_list)
```

<p style="text-align: justify">Finally, we can use the Parallel object to execute the list of tasks in parallel.</p>

```python
scores = executor(tasks)
```

<p style="text-align: justify">A non-parallel version of evaluating all model configurations is provided in case we want to debug something.</p>

```python
scores = [evaluate_arima_family_scores(func, name, order) for order in orders_list]
```

<p style="text-align: justify">The result of evaluating a list of configurations will be a list of tuples, each with a name that summarizes a specific model configuration and the error of the model evaluated with that configuration as either the RMSE or None if there was an error. We can filter out all scores with a None.</p>

```python
scores = [r for r in scores if r[1] != None]
```

<p style="text-align: justify">We can then sort all tuples in the list by the score in ascending order (best are first), then return this list of scores for review.</p>

<p style="text-align: justify">The <em>grid_search_arima_family()</em> function below implements this behavior given a univariate time series dataset, a specified model with a name (from a dictionary here), a list of model configurations (list of lists). An optional parallel argument allows the evaluation of models across all cores to be tuned on or off, and is on by default.</p>

In [8]:
# grid search configs for ARIMA model
def grid_search_arima_family(func, name, train, test, orders_list, parallel=True):
    scores = None
    if parallel:
        # execute configs in parallel
        executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
        tasks = (delayed(evaluate_arima_family_scores)(func, name, train, test, order) for order in orders_list)
        scores = executor(tasks)
    else:
        scores = [evaluate_arima_family_scores(func, name, train, test, order) for order in orders_list]
    # remove empty results
    scores = [r for r in scores if r[1] != None]
    # sort configs by error, asc
    scores.sort(key=lambda tup: tup[1])
    return scores, name

<p style="text-align: justify">The only thing left to do is to define a list of model configurations to try for a dataset.</p>

<p style="text-align: justify">For the ARIMA model, we create a function <em>arima_orders()</em> that will return a list containing the different orders that will be tested by the <em>grid_search_arima_family()</em> above.

<p style="text-align: justify">For the SARIMA model, we can define this generically. The only parameter we may want to specify is the periodicity of the seasonal component in the series, if one exists. By default, we will assume no seasonal component. The <em>Sarima_configs()</em> function below will create a list of model configurations to evaluate.</p>

In [9]:
# create a set of sarima configs to try
def arima_orders(p_values, d_values, q_values):
    orders = list()
    # create config instances
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                orders.append(order)
    return orders

def Sarima_configs(seasonal=[0]):
    configs = list()
    # define config lists
    p_params = [0, 1, 2]
    d_params = [0, 1]
    q_params = [0, 1, 2]
    t_params = ['n']
    P_params = [0, 1, 2]
    D_params = [0, 1]
    Q_params = [0, 1, 2]
    m_params = seasonal
    # create config instances
    for p in p_params:
        for d in d_params:
            for q in q_params:
                for t in t_params:
                    for P in P_params:
                        for D in D_params:
                            for Q in Q_params:
                                for m in m_params:
                                    cfg = [(p,d,q), (P,D,Q,m), t]
                                    configs.append(cfg)
    return configs

<p style="text-align: justify">Finally, the grid search procedure can be carried out by looping over the "<em>models</em>" dictionary, and the orders (configurations as arguments). The function below implements this procedure and print the 3 best configurations for each model after testing the different hyperparameters.</p>

```python
for (name, func), arg in zip(models.items(), args):
    scores, name = grid_search_arima_family(func, name, train, test, arg, parallel=False)
    print('\n')
    print('Top 3 best performing ARIMA models:')
    for order, error in scores[:3]:
        print(name + str(order), 'RMSE = ' + str(error))
```

<p style="text-align: justify">The script below shows the the full example with the importations of library, and some operations for loading, and pre-processing the data, which have been highlighted in the <a href="https://github.com/DavidCico/Univariate-time-series-analysis-of-cryptocurrency-data-with-ARIMA-and-SARIMA-and-hypergrid-search/blob/master/Univariate_analysis_classic_methods.ipynb">first notebook</a>. The different operations, such as splitting the dataset, defining the models and orders are then executed, before running the search grid for the models.</p>

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

from math import sqrt
from scipy.stats import boxcox
from statsmodels.graphics.gofplots import qqplot

from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

from warnings import catch_warnings
from warnings import filterwarnings
filterwarnings("ignore")

from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed


if __name__ == '__main__':
    # Write name of coin for study (XRP, BTC)
    name_coin = 'XRP'

    dataset = pd.read_csv(name_coin + '_price.csv', header=0, infer_datetime_format=True, parse_dates=['Date'], index_col=['Date'])

    # fill all NaN values with some particular value
    dataset.fillna(0, inplace=True)

    # make dataset numeric
    dataset = dataset.astype('float32')

    # look at the values of the dataset
    values = dataset.values

    n_test = 714 # Splitting the dataset 714 days before the end

    #for n_test in n_tests:
    train, test = split_dataset(dataset.values, n_test)

    # define the names and functions for the models we wish to evaluate
    models = dict()
    models['arima'] = arima_forecast
    models['Sarima'] = Sarima_forecast
    
    # values for the ARIMA orders range
    p_values = range(0, 8)
    d_values = range(0, 3)
    q_values = range(0, 2)

    orders_arima_list = arima_orders(p_values,d_values,q_values) # function to define ARIMA's orders list
    configs_sarima_list = Sarima_configs(seasonal=[0]) # fuction  for SARIMA configurations

    args = list() # list of arguments to be called in the loop
    args.extend([orders_arima_list, configs_sarima_list])

    for (name, func), arg in zip(models.items(), args): # loop on dictionary and configurations at same time
        scores, name = grid_search_arima_family(func, name, train, test, arg, parallel=False)
        print('\n')
        print('Top 3 best performing ARIMA models:')
        for order, error in scores[:3]: # loop ot print best 3 models of the tested function
            print(name + str(order), 'RMSE = ' + str(error)) 
        print('\n')

arima(0, 0, 0) RMSE=0.567
arima(0, 0, 1) RMSE=0.537
arima(0, 1, 0) RMSE=0.628
arima(0, 1, 1) RMSE=0.628
arima(0, 2, 0) RMSE=0.629
arima(0, 2, 1) RMSE=0.628
arima(1, 0, 0) RMSE=0.195
arima(1, 1, 0) RMSE=0.628
arima(1, 1, 1) RMSE=0.620
arima(1, 2, 0) RMSE=0.629
arima(1, 2, 1) RMSE=0.628
arima(2, 1, 0) RMSE=0.628
arima(2, 2, 0) RMSE=0.629
arima(2, 2, 1) RMSE=0.628
arima(3, 1, 0) RMSE=0.628
arima(3, 1, 1) RMSE=0.625
arima(3, 2, 0) RMSE=0.629
arima(3, 2, 1) RMSE=0.628
arima(4, 1, 0) RMSE=0.627
arima(4, 1, 1) RMSE=0.624
arima(4, 2, 0) RMSE=0.628
arima(5, 1, 0) RMSE=0.626
arima(5, 2, 0) RMSE=0.629
arima(6, 1, 0) RMSE=0.624
arima(6, 2, 0) RMSE=0.630
arima(6, 2, 1) RMSE=0.630
arima(7, 1, 0) RMSE=0.621
arima(7, 2, 0) RMSE=0.633
arima(7, 2, 1) RMSE=0.633


Top 3 best performing ARIMA models:
arima(1, 0, 0) RMSE = 0.19505876965089763
arima(0, 0, 1) RMSE = 0.537229162202763
arima(0, 0, 0) RMSE = 0.5672566568438188


Sarima[(0, 0, 0), (0, 0, 0, 0), 'n'] RMSE=0.629
Sarima[(0, 0, 0), (0, 0, 1, 0), 'n'

Sarima[(2, 1, 1), (2, 0, 1, 0), 'n'] RMSE=0.257
Sarima[(2, 1, 1), (2, 0, 2, 0), 'n'] RMSE=0.285
Sarima[(2, 1, 2), (0, 0, 0, 0), 'n'] RMSE=0.287
Sarima[(2, 1, 2), (0, 0, 1, 0), 'n'] RMSE=0.271
Sarima[(2, 1, 2), (0, 0, 2, 0), 'n'] RMSE=0.272
Sarima[(2, 1, 2), (1, 0, 0, 0), 'n'] RMSE=0.259
Sarima[(2, 1, 2), (1, 0, 1, 0), 'n'] RMSE=0.279
Sarima[(2, 1, 2), (1, 0, 2, 0), 'n'] RMSE=0.282
Sarima[(2, 1, 2), (2, 0, 0, 0), 'n'] RMSE=0.257
Sarima[(2, 1, 2), (2, 0, 1, 0), 'n'] RMSE=0.284
Sarima[(2, 1, 2), (2, 0, 2, 0), 'n'] RMSE=0.270


Top 3 best performing ARIMA models:
Sarima[(0, 1, 0), (0, 0, 0, 0), 'n'] RMSE = 0.1961476817757037
Sarima[(0, 1, 2), (0, 0, 1, 0), 'n'] RMSE = 0.19670211040324004
Sarima[(0, 1, 0), (0, 0, 1, 0), 'n'] RMSE = 0.19706578447768394




<p style="text-align: justify">The results show that the best models, for this case with weekly windows of 7 days are the <b>ARIMA(1,0,0)</b> and <b>SARIMA[(0, 1, 0), (0, 0, 0, 0), 'n']</b>. The seasonal component in our case is not obvious considering the short period of time for the windows. However, as seen in the top 3 best SARIMA models, the <b>Q</b> factor for the seasonal moving average plays a role for getting better results.</p>

<h2>Extensions</h2>
<p>This section lists some ideas for extending the tutorial that you may wish to explore.</p>
<ul>
<li><div style="text-align: justify"><strong>Data Transforms</strong>. Update the framework to support configurable data transforms such as normalization and standardization.</div></li>

<li><div style="text-align: justify"><strong>Changing rolling window period</strong>. Here, we used a rolling window of 7 days to predict the next 7 days of the cryptocurrency price. Depending on the problem encountered, there might be an optimal window in which the models need to be trained and re-defined for improved results.</div></li>

<li><div style="text-align: justify"><strong>Tune Amount of Historical Data</strong>. Update the framework to tune the amount of historical data used to fit the model.</div></li>

</ul>